In [1]:
import sys
import subprocess

exit_code = subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
if exit_code:
    raise Exception("Non-zero return code when installing the dependencies, please try reinstalling them manually")

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import boto3
import requests
ec2 = boto3.resource("ec2", "us-east-1")
group_name = "hillel-lesson20-2-security-group"

my_cidr = f'{requests.get("https://api.ipify.org?format=json").json().get("ip", "127.0.0.1")}/32'
print(my_cidr)

output = EC2_Client.create_security_group(
    Description="Security Group for Lesson 20",
    GroupName=group_name,
    VpcId="vpc-0565475cef4feac4e"
)

group_id = output.get('GroupId')
print("Successfully created security group")

ec2_client.authorize_security_group_ingress(
    GroupId=group_id,
    IpPermissions=[{
        "IpProtocol": "tcp",
        "FromPort": 80,
        "ToPort": 80,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "HTTP from my laptop"
        }]
    }, {
        "IpProtocol": "tcp",
        "FromPort": 22,
        "ToPort": 22,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "SSH from my laptop"
        }]
    }]
)

195.64.239.131/32


NameError: name 'EC2_Client' is not defined

In [22]:
import boto3

ami_id = "ami-0d263ea651113b610"

ec2_client = boto3.client("ec2", "us-east-1")


output = ec2_client.run_instances(
  MaxCount=1,
  MinCount=1,
  InstanceType="t3.micro",
  ImageId=ami_id,
  KeyName="hillel-gr",
  IamInstanceProfile={
    "Name": "hillel-lesson20-role"
  },
  SecurityGroupIds=[
    group_id,
  ],
  UserData="""
#!/usr/bin/env bash
docker run -d --name nginx -p 80:80 nginx:alpine
  """
)

instance_ids = list(map(lambda i : i.get("InstanceId"), output.get("Instances")))
print(f"Launched instances with ids: {' '.join(instance_ids)}")

jdun = ec2_client.get_waiter("instance_running")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are up and running")

Launched instances with ids: i-074195fbd35809f47
Instances with ids: i-074195fbd35809f47 are up and running


In [23]:
ec2_resource = boto3.resource("ec2", "us-east-1")
instance = ec2_resource.Instance(instance_ids[0])
print(instance.public_ip_address)

34.200.219.71


**Terminate** instances and wait until they are down:

In [24]:
ec2_client.terminate_instances(InstanceIds=instance_ids)
jdun = ec2_client.get_waiter("instance_terminated")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are terminated")

Instances with ids: i-074195fbd35809f47 are terminated
